In [ ]:
#Proceso01
#se va a trabajar sobre todas las columnas para realizar una primera exploracion de los datos.
#se va a generar un nuevo csv como salida de este proceso para poder trabajarlo posteriormente

In [ ]:
#importando librerias que se piensan utilizar
import numpy as np
import pandas as pd
import seaborn as sns
import unidecode
import re
import pandas as pd

In [ ]:
#
# funciones comunes
#

#funcion para normalizar texto a unicode y a minuscula
def normalizar_string(texto):
    return str.lower(unidecode.unidecode(texto))

In [ ]:
#leyedo datos de properatti.csv
propiedades = pd.read_csv('../data/properatti.csv', decimal=".")

#renombrado primer columana que no tiene nombre
propiedades.rename(columns={'Unnamed: 0':'id'}, inplace=True)

#tamaño y forma del df
propiedades.shape

In [ ]:
propiedades.columns

In [ ]:
propiedades.head()

In [ ]:
#
#'id'
#

#verificamos si id no esta repetido para poder utilizarlo como id unico.
#si el count no es 0 hay duplicados.
dfTemp = propiedades[['id']].copy()
dfTemp[dfTemp['id'].duplicated(keep=False)].count() > 1

In [ ]:
#nuevo df para ir guardando los datos procesados.
dfNuevo = dfTemp

In [ ]:
#
#'operation'
#

#agrupamos para ver que tipo de operaciones hay y si alguna fila que no tiene tipo asignado
dfTemp = propiedades[['id','operation']].copy()
dfTemp.groupby('operation').count()

In [ ]:
#se le suma una columna pais con valor argentina al df.
dfNuevo['operacion'] = 'venta'

In [ ]:
#
#'property_type'
#

#agrupamos para ver que tipo de propiedades hay y si alguna fila que no tiene tipo asignado
dfTemp = propiedades[['id','property_type']].copy()
dfTemp.groupby('property_type').count()

In [ ]:
#diccionario de palabras ingles/castellano
tipo_propiedades = {
  'ph': 'ph',
  'apartment': 'departamento',
  'house': 'casa',
  'store': 'local'
}

#se pasa todo a minuscula
dfTemp['property_type'] = dfTemp['property_type'].map(str.lower)

#se genera nueva columna con el valor que tomamos del diccionanio.
dfTemp['tipo_propiedad'] = dfTemp['property_type'].map(tipo_propiedades)

#se elimina la columna con datos en ingles y la columna id usada para el count.
dfTemp.drop(['id'], axis=1, inplace=True)
dfTemp.drop(['property_type'], axis=1, inplace=True)

#se suma al df las columnas tipo_propiedad
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo['pais'] = 'argentina'

In [ ]:
#agrupamos para ver que paises hay y si alguna fila que no tiene tipo asignado
dfTemp = propiedades[['id','country_name']].copy()
dfTemp.groupby('country_name').count()

In [ ]:
#
#'property_type'
#

#se le suma una columna pais con valor argentina al df.
dfNuevo['pais'] = 'argentina'

In [ ]:
dfNuevo.head()

In [ ]:
#
# 'place_with_parent_names'
#

#se separa contenido de la columna place_with_parent_names
dfTemp = pd.DataFrame(propiedades['place_with_parent_names'].str.split('|', expand=True))

#se reemplaza espacios vacios (None) por string vacios
dfTemp.fillna(value='', inplace=True)

#se reemplaza string vacio por np.nan
dfTemp.replace(to_replace='', value=np.nan, inplace=True)

#se detectar que columna no contiene datos y se elimina -> vacia es igual a 121219
for column in dfTemp.columns:
    datos = dfTemp[column].isnull().sum()
    if datos == 121219:
        print("columna",column,"vacia")
        dfTemp.drop([column], axis=1, inplace=True)
    else:
        print("columna",column,"con",121219-datos,"registros")

#se vuelven a poner los campos como string vacios
dfTemp.replace(to_replace=np.nan, value='', inplace=True)

#se normaliza el contenido de las columnas string
dfTemp = pd.DataFrame(dfTemp.applymap(normalizar_string))

dfTemp.head(20)

In [ ]:
#se elimina la columna pais que ya fue cargada
dfTemp.drop([1], axis=1, inplace=True)

#se le asigna nombre a las columnas
dfTemp.columns=['provincia', 'ciudad_barrio', 'barrio', 'barrio_cerrado']

dfNuevo.head(20)

In [ ]:
#
#place_name
#

#se suma place_name al dfTemp para trabajar en conjunto con las columnas de place_with_parent_names.
#se asume que place_name puede ser un barrio o una ciudad.
dfTemp = dfTemp.join(propiedades['place_name'])
dfTemp.replace(to_replace=np.nan, value='', inplace=True)

#se normaliza el contenido de las columnas string
dfTemp = pd.DataFrame(dfTemp.applymap(normalizar_string))

#se toma ciudad_barrio como valor correcto en caso de estar, sobre el contenido de place_name
dfTemp = dfTemp.join(pd.DataFrame(dfTemp.apply(lambda x: x['ciudad_barrio'] if(x['ciudad_barrio']!='') else ('' if(x['place_name'] == 'capital federal') else x['place_name']), axis=1)))

#se eliminana columna ya procesada 
dfTemp.drop(['ciudad_barrio'], axis=1, inplace=True)

#se renombran la columna generada e el join como nueva ciudad_barrio
dfTemp.rename(columns={0: 'ciudad_barrio'}, inplace=True)

dfTemp.head()

In [ ]:
#
#barrio
#

#si provincia es capital_federal y ciudad_barrio es igual a barrio se asigna string vacio a barrio.
#si place_name es igual a ciudad_barrio y barrio es vacio se asigna string vacio a barrio.
#si barrio es vacio ponemos el valor de place
#sino dejamos el valor de barrio.
dfTemp = dfTemp.join(
    pd.DataFrame(
        dfTemp.apply(lambda x: '' 
                     if(x['provincia']=='capital federal' and x['ciudad_barrio']==x['barrio'])
                     else('' if(x['place_name']==x['ciudad_barrio'] and x['barrio']=='') 
                          else(x['place_name'] 
                               if(x['barrio']=='') 
                               else x['barrio']
                              )
                         ), axis=1
                    )
    )
)

#
#ejemplo
#
#['capital federal', '', 'barrio_cerrado', 'palermo', 'capital federal'] -> '' (aplica regla 1, no tiene barrio)
#['capital federal', '', 'barrio_cerrado', 'palermo', 'palermo']  -> '' (aplica regla 2, para no repetir palermo)
#['capital federal', '', 'barrio_cerrado', 'palermo soho', 'palermo'] -> 'palermo soho' (aplica regla 3, sabemos 
#que no va a ser capital federal, savemos que no repite barrio)
#['capital federal', 'palermo soho', 'barrio_cerrado', 'palermo', 'palermo', ''] -> 'palermo soho'

#se eliminana columnas ya procesadas o que no se van a utilizar
dfTemp.drop(['barrio'], axis=1, inplace=True)
dfTemp.drop(['place_name'], axis=1, inplace=True)

#se renombran la columna 0 a barrio
dfTemp.rename(columns={0: 'barrio'}, inplace=True)

#se suma al df las columnas completas y las nuevas barrio y barrio_cerrado
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head(20)

In [ ]:
#
#barrio_cerrado
#

#se lleva la columna barrio_cerrado al final del df
dfTemp = dfNuevo.pop('barrio_cerrado')

dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head(20)

In [ ]:
#
# transformar 'geonames_id' en una URL
#
# TODO: Enriquecer información geográfica de la ciudad/barrio/comuna para poder compararla con 
# los datos ya provistos y sanear la ubicación geográfica.

#df temporal es para poder aplicarle una funciona lambda, transformo el dato a integer y reemplazo NaN a 0
dfTemp = propiedades[['geonames_id']].copy()
dfTemp.replace(to_replace=np.nan, value=0, inplace=True)
dfTemp = dfTemp.astype(np.int64)

#se arma la url de geonames sumando el id
dfTemp = pd.DataFrame(dfTemp.apply(lambda x: 'http://www.geonames.org/'+str(x['geonames_id']) if(x['geonames_id']!=0) else '', axis=1))

#se renombran la columna nueva
dfTemp.columns=['geonames_url']

#se suma al df la nueva columna geonames_url
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#funcion para verificar lat y lon
#

#patrones
patron_lat = re.compile('^-?([1-8]?[1-9]|[1-9]0)\.{1}\d{1,6}')
patron_lon = re.compile('^-?([1]?[1-7][1-9]|[1]?[1-8][0]|[1-9]?[0-9])\.{1}\d{1,6}')

def verificar_lat(lat):
    var = True
    
    if(patron_lat.match(lat)==None):
        var = False
    elif(float(lat)>0):
        #por estar bajo el ecuador
        var = False
        
    return var

def verificar_lon(lon):
    var = True
    
    if(patron_lon.match(lon)==None):
        var = False
    elif(float(lon)>0):    
        #por estar al oeste de greenwich
        var = False
        
    return var

In [ ]:
#
#'lat','lon'
#

dfTemp = propiedades[['lat','lon']].copy()

dfTemp = dfTemp.join(propiedades['lat-lon'].str.split(',', expand=True))

#se renombran la columna generada en el join como nueva ciudad_barrio
dfTemp.rename(columns={0: 'latsplit'}, inplace=True)
dfTemp.rename(columns={1: 'lonsplit'}, inplace=True)

print(dfTemp.count())

dfTemp.head(5)

In [ ]:
#se considera los valores de ['lat-lon'] como validos. (parecen ser similares a simple vista y 
#cantidad de registros)
#verifican los nulos en latsplit y se completan con los de las columnas separadas.

dfTemp['nueva_lat'] = dfTemp[['lat', 'latsplit']].apply(lambda x: x['latsplit'] 
                                                        if(not np.isnan(float(x['latsplit']))) 
                                                        else x['lat'], axis=1
                                                       )

dfTemp['nueva_lon'] = dfTemp[['lon', 'lonsplit']].apply(lambda x: x['lonsplit'] 
                                                        if(not np.isnan(float(x['lonsplit']))) 
                                                        else x['lon'], axis=1
                                                       )

#nos quedamos con los nuevos valores
dfTemp = dfTemp[['nueva_lat','nueva_lon']]

dfTemp.head()

In [ ]:
#se reemplaza string vacio por np.nan y pasamos a tipo string para poder evaluarlos con una regex
dfTemp.replace(to_replace=np.nan, value='', inplace=True)
dfTemp = dfTemp.astype('str')

#verificamos si lat son validas
dfTemp = dfTemp.join(pd.DataFrame(dfTemp.apply(lambda x: x['nueva_lat'] 
                                               if(verificar_lat(x['nueva_lat'])) 
                                               else np.nan, axis=1
                                              )
                                 )
                    )

dfTemp.drop(['nueva_lat'], axis=1, inplace=True)
dfTemp.rename(columns={0: 'lat'}, inplace=True)

#verificamos si lon son validas
dfTemp = dfTemp.join(pd.DataFrame(dfTemp.apply(lambda x: x['nueva_lon'] 
                                               if(verificar_lon(x['nueva_lon'])) 
                                               else np.nan, axis=1
                                              )
                                 )
                    )

dfTemp.drop(['nueva_lon'], axis=1, inplace=True)
dfTemp.rename(columns={0: 'lon'}, inplace=True)

#se suma al df las columnas lat lon
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#'precio'
#
# TODO: ver que pasa si price_aprox_usd no tiene valor en dolares.

#df temporal es para poder formatear (price)  
dfTemp = propiedades[['price_aprox_usd']].copy()

#se renombran la columna a precio
dfTemp.columns=['precio']

#se suma al df la columna precio
dfNuevo = dfNuevo.join(dfTemp)

In [ ]:
#
#'currency'
#

#agrupamos para ver que monedas hay y si alguna fila que no tiene tipo asignado
dfTemp = propiedades[['id','currency']].copy()
dfTemp.groupby('currency').count()

In [ ]:
#se crea la columna moneda y se le asigna valor usd ya que vamos a trabajar en dolares.
dfNuevo['moneda'] = 'usd'

dfNuevo.head()

In [ ]:
#
#'total_m2','total_m2_cubiertos','total_m2_decubiertos'
#

#df temporal es para poder aplicarle una funciona lambda
dfTemp = propiedades[['surface_total_in_m2','surface_covered_in_m2']].copy()

#si surface_total_in_m2 y surface_covered_in_m2 son distintos de 0 y surface_total_in_m2 mayor/igual 
#surface_covered_in_m2 se calcula la superficie descubierta.
dfTemp = dfTemp.join(pd.DataFrame(dfTemp.apply(lambda x: 
                                               x['surface_total_in_m2']-x['surface_covered_in_m2'] 
                                               if(x['surface_total_in_m2'] > x['surface_covered_in_m2']
                                                  and x['surface_covered_in_m2'] > 0) 
                                               else np.nan, axis=1)
                                 )
                    )

dfTemp.columns=['sup_m2','sup_m2_cubiertos','sup_m2_decubiertos']
    
#se suma al df las columnas total_m2, total_m2_cubiertos, total_m2_decubiertos(nueva)
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#nueva columna valor_m2
#

#df temporal es para poder aplicarle una funciona lambda
dfTemp = dfNuevo[['precio','sup_m2']].copy()

#division para tener el valor del m2
#si precio y total_m2 son mayor 0 se realiza la division.
#sino no se asigna np.nan al valor del m2
dfTemp = pd.DataFrame(dfTemp.apply(lambda x: x['precio']/x['sup_m2'] 
                                   if(x['precio'] > 0 and x['sup_m2'] > 0 and x['precio'] > x['sup_m2']) 
                                   else np.nan, axis=1
                                  )
                     )

dfTemp.columns=['valor_m2']

#se suma al df la nueva columna valor_m2
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#floor
#

#df temporal es para poder trabajar sombre la columna floor
dfTemp = propiedades[['property_type','floor']].copy()

#mayores a 54 se considera no informado. No hay edificios con mas de 54 pisos en argentina.
dfTemp.loc[dfTemp['floor'] > 54] = 'mal informado'

#se reemplazo NaN a 'no informado'
dfTemp.replace(to_replace=np.nan , value='no informado', inplace=True)

#Casa no requiere informar piso 'No requerido'
dfTemp = pd.DataFrame(dfTemp.apply(lambda x: x['floor'] 
                                   if(x['property_type']!='house') 
                                   else 'no requerido', axis=1
                                  )
                     )

dfTemp.columns=['piso']

#se suma al df la columna piso
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#rooms
#

#df temporal es para poder trabajar sombre la columna rooms
dfTemp = propiedades[['rooms']].copy()

#se reemplazo NaN a 'no informado'
dfTemp.replace(to_replace=np.nan , value='no informado', inplace=True)
dfTemp.replace(to_replace='' , value='no informado', inplace=True)

dfTemp.columns=['habitaciones']

#se suma al df la columna habitaciones
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#expenses
#

#df temporal es para poder trabajar sombre la columna expenses
dfTemp = propiedades[['expenses']].copy()

dfTemp.columns=['expensas']

#se suma al df la columna expensas
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#properati_url
#
# TODO: ver qué se puede obtener con este feature

In [ ]:
#
#title
#

#df temporal para renombrarar columna 
dfTemp = pd.DataFrame(propiedades[['title']])

#se reemplazo NaN a 'no informado'
dfTemp.replace(to_replace=np.nan , value='no informado', inplace=True)

#se normaliza el contenido de las columnas string
dfTemp = pd.DataFrame(dfTemp.applymap(normalizar_string))
dfTemp.columns=['titulo']

#se suma al df la columna titulo
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
#
#description
#

#df temporal para renombrarar columna 
dfTemp = pd.DataFrame(propiedades[['description']])

#se reemplazo NaN a 'no informado'
dfTemp.replace(to_replace=np.nan , value='no informado', inplace=True)

#se normaliza el contenido de las columnas string
dfTemp = pd.DataFrame(dfTemp.applymap(normalizar_string))
dfTemp.columns=['descripcion']

#se suma al df la columna descripcion
dfNuevo = dfNuevo.join(dfTemp)

dfNuevo.head()

In [ ]:
dfNuevo.columns

In [ ]:
#se guarda csv completo
dfNuevo.to_dense().to_csv("../data/out_proceso01.csv", index = False, sep=',', encoding='utf-8')